In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
from nltk.tokenize import wordpunct_tokenize
import string

In [3]:
!cp ./drive/'My Drive'/gender/df_final.csv .

In [4]:
dataset = pd.read_csv('df_final.csv',index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
dataset

,NOMBRE,SEXO
0,AARON,M
1,AARONIT,M
2,ABA,F
3,ABACO,M
4,ABALEN,M
...,...,...
676595,Shelly,F
676596,Tory,F
676597,Batsheva,F
676598,Chelsie,F


In [6]:
dataset = dataset.drop(dataset.index[dataset['SEXO']=='A'])

In [7]:
len(dataset['NOMBRE'].unique())

84337

In [8]:
dataset = dataset.drop_duplicates(subset='NOMBRE',ignore_index=True)

In [9]:
dataset.isnull().values.any()

True

In [10]:
dataset=dataset.dropna()

In [11]:
dataset.isnull().values.any()

False

In [12]:
dataset = dataset.drop(5784)

In [13]:
dataset['NOMBRE'] = dataset['NOMBRE'].str.lower()

In [14]:
rows_list = [wordpunct_tokenize(i) for i in dataset['NOMBRE']]

In [15]:
def clean(list_rows):
  tilde = {
      'á':'a',
      'é':'e',
      'í':'i',
      'ó':'o',
      'ú':'u'
  }
  punctuation = string.punctuation
  new_rows =[]
  for list_words in list_rows:
    new_list_words = []
    for word in list_words:
      clean_word = ''
      
      for letter in word:
        if letter in tilde.keys():#quitar tildes
          letter = tilde[letter]
        if letter not in punctuation:  
          clean_word+=letter
      if (len(clean_word)>1):
        new_list_words.append(clean_word)
    new_rows.append(new_list_words)
  return new_rows

In [16]:
new_row_clean = clean(rows_list)

In [17]:
dataset['clean_names'] = [' '.join(word) for word in [word_list for word_list in new_row_clean]]

In [18]:
name=dataset['clean_names']

In [19]:
len(name.unique())

73566

In [20]:
name=list(name)

In [21]:
label=list(dataset['SEXO'])

In [22]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [23]:
labels=le.fit_transform(label)

In [24]:
# labels

In [25]:
le.classes_

array(['F', 'M'], dtype='<U1')

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

In [27]:
cv=CountVectorizer(analyzer='char')

In [28]:
names=cv.fit_transform(name).toarray()

In [29]:
len(names[1])

53

In [30]:
from sklearn.model_selection import train_test_split
feature_train,feature_test,label_train,label_test=train_test_split(names,labels,test_size=0.2,random_state=42)

In [31]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(feature_train,label_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [32]:
label_pred=model.predict(feature_test)

In [33]:
import sklearn.metrics as m

In [34]:
m.accuracy_score(label_test,label_pred)

0.7010731013221082

LSTM

In [35]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(name)
sequence_of_int = tokenizer.texts_to_sequences(name)


In [36]:
from keras.preprocessing.sequence import pad_sequences
padsequences=pad_sequences(sequence_of_int,maxlen=80,padding='post')

In [37]:
len(padsequences[2])

80

In [38]:
from keras.utils.np_utils import to_categorical
labels=to_categorical(labels)

In [39]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding,Dropout

In [40]:
padsequences.shape

(84335, 80)

In [41]:
padsequences.shape

(84335, 80)

In [42]:
padsequences = np.reshape(padsequences,(padsequences.shape[0],padsequences.shape[1],1))#esto add yo

In [43]:
from sklearn.model_selection import train_test_split
feature_train,feature_test,label_train,label_test=train_test_split(padsequences,labels,test_size=0.1,random_state=42)

In [44]:
print(feature_train.shape)
print(feature_test.shape)
print(label_train.shape)
print(label_test.shape)

(75901, 80, 1)
(8434, 80, 1)
(75901, 2)
(8434, 2)


In [45]:
label_test[0]

array([1., 0.], dtype=float32)

In [46]:
27*64

1728

In [47]:
max(feature_train.ravel())

53

In [52]:
model=Sequential()
model.add(Embedding(53+1,100,input_length=80))
model.add(LSTM(2048,return_sequences=True))
model.add(LSTM(256,return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(2,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 80, 100)           5400      
_________________________________________________________________
lstm_4 (LSTM)                (None, 80, 2048)          17604608  
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               2360320   
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 514       
Total params: 19,970,842
Trainable params: 19,970,842
Non-trainable params: 0
_________________________________________________________________


In [53]:
model.fit(feature_train,label_train,batch_size=1000,epochs=50,validation_data=(feature_test,label_test),verbose=1)


Epoch 1/50
76/76 [==============================] - 271s 4s/step - loss: 0.6937 - accuracy: 0.5030 - val_loss: 0.6933 - val_accuracy: 0.4948
Epoch 2/50
76/76 [==============================] - 272s 4s/step - loss: 0.7072 - accuracy: 0.5047 - val_loss: 0.6929 - val_accuracy: 0.5154
Epoch 3/50
76/76 [==============================] - 273s 4s/step - loss: 0.6899 - accuracy: 0.5289 - val_loss: 0.6887 - val_accuracy: 0.5500
Epoch 4/50
76/76 [==============================] - 273s 4s/step - loss: 0.6842 - accuracy: 0.5559 - val_loss: 0.6838 - val_accuracy: 0.5612
Epoch 5/50
76/76 [==============================] - 272s 4s/step - loss: 0.6833 - accuracy: 0.5588 - val_loss: 0.6845 - val_accuracy: 0.5708
Epoch 6/50
76/76 [==============================] - 273s 4s/step - loss: 0.6789 - accuracy: 0.5720 - val_loss: 0.6742 - val_accuracy: 0.5816
Epoch 7/50
76/76 [==============================] - 273s 4s/step - loss: 0.6871 - accuracy: 0.5384 - val_loss: 0.6916 - val_accuracy: 0.4948
Epoch 8/50
76

In [54]:
model.save('model.h5')

In [55]:
import sklearn.metrics as m


In [56]:
label_pred=model.predict(feature_test)

In [57]:
label_pred=np.argmax(label_pred,axis=1)

In [58]:
label_pred

array([1, 1, 0, ..., 1, 0, 1])

In [59]:
l_test=np.argmax(label_test,axis=1)

In [60]:
m.accuracy_score(l_test,label_pred)

0.9047901351671804

In [61]:
print(m.classification_report(l_test,label_pred))

              precision    recall  f1-score   support

           0       0.91      0.90      0.91      4261
           1       0.90      0.91      0.90      4173

    accuracy                           0.90      8434
   macro avg       0.90      0.90      0.90      8434
weighted avg       0.90      0.90      0.90      8434



In [62]:
m.confusion_matrix(l_test,label_pred)

array([[3850,  411],
       [ 392, 3781]])

In [63]:
from keras.models import load_model

In [64]:
import pickle

In [65]:
pickle.dump(tokenizer,open('tokenizer.pkl','wb+'),protocol=pickle.HIGHEST_PROTOCOL)

In [66]:
tokenizer=pickle.load(open('tokenizer.pkl','rb+'))

In [67]:
tokenizer.index_word

{1: 'a',
 2: 'i',
 3: 'e',
 4: 'n',
 5: 'r',
 6: 'o',
 7: 'l',
 8: ' ',
 9: 's',
 10: 'm',
 11: 't',
 12: 'd',
 13: 'u',
 14: 'c',
 15: 'g',
 16: 'h',
 17: 'b',
 18: 'j',
 19: 'y',
 20: 'v',
 21: 'f',
 22: 'p',
 23: 'k',
 24: 'z',
 25: 'w',
 26: 'x',
 27: 'q',
 28: 'ð',
 29: 'ö',
 30: 'þ',
 31: 'ñ',
 32: 'ü',
 33: 'ý',
 34: 'ç',
 35: 'æ',
 36: 'ë',
 37: 'è',
 38: 'ş',
 39: 'ğ',
 40: 'ı',
 41: 'ä',
 42: 'ï',
 43: 'à',
 44: 'å',
 45: 'ò',
 46: 'ì',
 47: 'ã',
 48: 'ô',
 49: 'ι',
 50: 'â',
 51: 'ù',
 52: 'î',
 53: 'ê'}

In [75]:
def input(n):
  q=tokenizer.texts_to_sequences(n)
  q=[i[0] for i in q]
  q1=[]
  q1.append(q)
  s=pad_sequences(q1,maxlen=80,padding='post')
  return s

In [76]:
tokenizer.texts_to_sequences('cristhian')

[[14], [5], [2], [9], [11], [16], [2], [1], [4]]

In [77]:
input('kamesh')

array([[23,  1, 10,  3,  9, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
      dtype=int32)

In [78]:
def pred(n):
  w=model.predict(input(n))
  w=np.argmax(w,axis=1)
  return w[0]

In [79]:
pred('josefina')

0

In [81]:
pred('cristhian')

1

In [82]:
pred('maria')

0

In [83]:
pred('jose')

1

In [84]:
pred('alex')

1

In [ ]:
pred('preethi')

In [ ]:
pred('John')

In [ ]:
pred('mahesh')

In [ ]:
pred('raju')

In [ ]:
pred('abhitha')

In [ ]:
pred('ratnamahadevi')

In [ ]:
pred('srimukhi')

In [ ]:
pred('rajashekhar')

In [ ]:
pred('manga')

In [ ]:
pred('katyayani')

In [ ]:
pred('prakash')

In [ ]:
pred('shanmukh')

In [ ]:
pred('bhargav')

In [ ]:
pred('aditya')

In [ ]:
pred('akshar')

In [ ]:
pred('madhav')

In [ ]:
pred('madhavi')

In [ ]:
pred('durga')

In [ ]:
pred('saisudha')

In [ ]:
pred('durgesh')

In [ ]:
pred('rangesh')

In [ ]:
pred('rangarao')

In [ ]:
pred('kameshwari')

In [ ]:
pred('kameswararao')

In [ ]:
model.save('model.h5')

In [ ]:
pred('srilekha')

In [ ]:
pred('srikhar')

In [ ]:
pred('aliabhatt')

In [ ]:
pred('aryabatt')

In [ ]:
pred('harsh')

In [ ]:
pred('garimasingh')

In [ ]:
pred('prabhkiratsingh')

In [ ]:
pred('rakesh')

In [ ]:
pred('uday')

In [ ]:
pred('krishna')

In [ ]:
pred('ram')

In [ ]:
pred('satyasai')

In [ ]:
pred('divyansh')

In [ ]:
pred('anushka')

In [ ]:
pred('kajol')

In [ ]:
pred('rithvik')

In [ ]:
pred('AlamKhan')

In [ ]:
pred('revathi')

In [ ]:
pred('deepika')

In [ ]:
pred('bunty')

In [ ]:
pred('pandu')